In [1]:
#read validation results
import pandas as pd
path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/ML based speckle POC/"
model_path = path +'ModelBuilding/MergedData_TrainValExchange/WithTopHVQKDiffFs/'
val_results =  pd.read_csv(model_path+"Val_results.csv") 

In [2]:
val_results["Set"]=2 #label all dies as set 2
df_set1 = pd.read_csv(path + "DataPreparation/SNR_R5_ww51.4/SupFeatures_Val.csv")
val_results.loc[val_results["VID"].isin(df_set1["VID"]), 'Set'] = 1 #replace label for dies in set 1 as 1
val_results

,VID,SORTLOT,WAFER,XLOC,YLOC,TEST RESULTS,TEST RESULTS BITS,TR_BITS,INCOMING,INCOMING BITS,...,HVQK_VMIN_DIFF_ARRAY_UNCORE_CABIST_SSALLCOTHRS_132110,HVQK_VMIN_DIFF_ARRAY_UNCORE_CABIST_SSALLCTAG_132110,HVQK_VMIN_DIFF_ARRAY_UNCORE_CABIST_SSASF_132110,HVQK_VMIN_DIFF_ARRAY_UNCORE_MBIST_LSAHQM_132110,HVQK_VMIN_DIFF_ARRAY_UNCORE_MBIST_LSAMEMCTRL_132110,HVQK_VMIN_DIFF_SCAN_UNCORE_LLCSFP11_132110,HVQK_VMIN_DIFF_SCAN_UNCORE_LLCSFP12_132110,HVQK_VMIN_DIFF_SCAN_UNCORE_LLCSFP23_132110,SPECKLE,PRED_SPECKLE
0,M0118LB500644,N0293010,594,3,3,B000100,100,1.0,B000000,0.0,...,0.000000,0.020132,0.043246,0.036158,0.036272,0.136252,0.032002,0.220844,1,1
1,M0310QS400090,N0284310,248,-1,-8,B100100,100100,2.0,B100000,100000.0,...,0.000000,0.021637,0.115138,0.026731,0.126441,0.026718,0.040364,0.144712,1,1
2,M0310QS400093,N0284310,248,0,-5,B001100,1100,2.0,B000000,0.0,...,0.000000,0.177429,0.043381,0.036352,0.116661,0.024744,0.037673,0.134126,1,1
3,M038791100022,N0284200,295,-4,-8,B000011,11,2.0,B000001,1.0,...,0.000000,0.034484,0.039579,0.031929,0.111771,0.020971,0.032522,0.113718,1,1
4,M03B8Q3300001,N0371390,958,6,-2,B000011,11,2.0,B000010,10.0,...,0.105736,0.012825,0.019874,0.071452,0.069996,0.013469,0.028674,0.100731,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15611,M1KN334000252,N030398,214,-3,-2,B000000,0,0.0,B000000,0.0,...,0.000000,0.015312,0.023619,0.044887,0.086343,0.015662,0.031481,0.088944,0,0
15612,M0XP855501789,N026648,80,-2,-1,B000000,0,0.0,B000000,0.0,...,0.000000,0.012542,0.019435,0.052608,0.068180,0.013227,0.022598,0.077704,0,0
15613,M0NQ816200202,N028634,578,-3,-2,B000000,0,0.0,B000000,0.0,...,0.000000,0.014374,0.022236,0.046866,0.080195,0.014850,0.024652,0.107061,0,0
15614,M1NT416600129,N038309,266,3,6,B000000,0,0.0,B000000,0.0,...,0.000000,0.008725,0.012147,0.051599,0.065805,0.008774,0.017361,0.076176,0,0


In [ ]:
# val_results["Set"]=2 #label all dies as set 2
# df = pd.read_csv(path + "DataPreparation/MergedData_WithC0stepping/SupFeatures_Val.csv")
# df_set1 = df[df["Set"]==1]
# df_set3 = df[df["Set"]==3]
# val_results.loc[val_results["VID"].isin(df_set1["VID"]), 'Set'] = 1 #replace label for dies in set 1 as 1
# val_results.loc[val_results["VID"].isin(df_set3["VID"]), 'Set'] = 3
# val_results

In [3]:
val_results["Set"].value_counts()

2    10876
1     4740
Name: Set, dtype: int64

In [4]:
val_results["SPECKLE"].value_counts()

0    13891
1     1725
Name: SPECKLE, dtype: int64

In [5]:
def val_result_split(val,set_num):
    
    val_set = val[val["Set"]==set_num]
    y_val = val_set["SPECKLE"]
    y_pred = val_set["PRED_SPECKLE"]
    
    import numpy as np
    from sklearn import metrics

    class_accuracies = []

    for class_ in y_val.sort_values(ascending= True).unique():
        class_acc = round(np.mean(y_pred[y_val == class_] == class_)*100,2)
        class_accuracies.append(class_acc)

    class_acc = pd.DataFrame(class_accuracies,index=['true:0', 'true:1'],columns= ["Accuracy %"])

    cf_matrix = pd.DataFrame(
        metrics.confusion_matrix(y_val, y_pred, labels= [0, 1]), 
        index=['true:0', 'true:1'], 
        columns=['pred:0', 'pred:1']
    )

    ascend = None #input None/True/False to order the confusion matrix
    if ascend == None:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
    else:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)

    #change as required
    cf_matrix.to_csv(model_path + "Confusion_matrix_set"+ str(set_num) + ".csv") 

    display(cf_matrix)


In [6]:
set_num = 1 #confusion matrix for set 1 will be printed in model_path
val_result_split(val_results,set_num)

,pred:0,pred:1,Accuracy %
true:0,3626,577,86.27
true:1,126,411,76.54


In [7]:
set_num = 2 #confusion matrix for set 2 will be printed in model_path
val_result_split(val_results,set_num)

,pred:0,pred:1,Accuracy %
true:0,8212,1476,84.76
true:1,387,801,67.42


In [ ]:
set_num = 3#confusion matrix for set 1 will be printed in model_path
val_result_split(val_results,set_num)

In [8]:
val_results.to_csv(model_path+"Val_results.csv",index=False) #save val_results with label